In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# Model configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Load Model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token=""  # Replace with your token
)

==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# using LORA Adapters

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                       # LoRA rank
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2025.3.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


# defining alpaca Prompt

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["Instruction"]
    outputs = examples["Answer"]

    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, "", output) + EOS_TOKEN
        texts.append(text)

    return { "text": texts }
pass

# load and split Dataset

In [7]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/output.json")

dataset = dataset["train"].train_test_split(test_size=0.2, seed=3407)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

train_dataset = train_dataset.map(formatting_prompts_func, batched=True, num_proc=2)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched=True, num_proc=2)

Map (num_proc=2):   0%|          | 0/460 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/115 [00:00<?, ? examples/s]

In [8]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(eval_dataset)}")

Train dataset size: 460
Validation dataset size: 115


# Supervised Fine-Tuning Trainer  initializing

In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 3,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 20,
        max_steps = 160,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        evaluation_strategy="steps",
        eval_steps=20,
    ),
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing to ["text"] (num_proc=3):   0%|          | 0/460 [00:00<?, ? examples/s]

Tokenizing to ["text"] (num_proc=3):   0%|          | 0/115 [00:00<?, ? examples/s]

# start fine tuning

In [10]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 460 | Num Epochs = 12 | Total steps = 160
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 29,933,568/1,830,055,936 (1.64% trained)


Step,Training Loss,Validation Loss
20,2.680300,2.013773
40,1.239000,1.190110
60,1.007400,1.062270
80,0.937300,1.013381
100,0.851800,0.989729
120,0.800700,0.981624
140,0.773800,0.981477
160,0.769300,0.981282


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=160, training_loss=1.3132228642702102, metrics={'train_runtime': 734.2443, 'train_samples_per_second': 6.973, 'train_steps_per_second': 0.218, 'total_flos': 4917818704232448.0, 'train_loss': 1.3132228642702102})

In [11]:
model.save_pretrained_gguf("model",tokenizer,quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.69 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 36/36 [00:02<00:00, 17.99it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {2048

In [15]:
!ls -lh /content/model


total 14G
-rw-r--r-- 1 root root  605 Mar  9 15:08 added_tokens.json
-rw-r--r-- 1 root root  808 Mar  9 15:08 config.json
-rw-r--r-- 1 root root  266 Mar  9 15:08 generation_config.json
-rw-r--r-- 1 root root 1.6M Mar  9 15:08 merges.txt
-rw-r--r-- 1 root root 4.7G Mar  9 15:09 pytorch_model-00001-of-00002.bin
-rw-r--r-- 1 root root 1.2G Mar  9 15:09 pytorch_model-00002-of-00002.bin
-rw-r--r-- 1 root root  35K Mar  9 15:09 pytorch_model.bin.index.json
-rw-r--r-- 1 root root  614 Mar  9 15:08 special_tokens_map.json
-rw-r--r-- 1 root root 7.2K Mar  9 15:08 tokenizer_config.json
-rw-r--r-- 1 root root  11M Mar  9 15:08 tokenizer.json
-rw-r--r-- 1 root root 5.8G Mar  9 15:13 unsloth.F16.gguf
-rw-r--r-- 1 root root 1.8G Mar  9 15:19 unsloth.Q4_K_M.gguf
-rw-r--r-- 1 root root 2.7M Mar  9 15:08 vocab.json


# inference script

In [12]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is HAI-LLM ?", # instruction
        "", # input  - leave this blank
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nwhat is HAI-LLM ?\n\n### Input:\n\n\n### Response:\nHigh-Performance AI Language Model<|im_end|>']

# Drive upload gguf file

In [16]:
from google.colab import drive
drive.mount('/content/drive')
import shutil


model_path = '/content/model/unsloth.Q4_K_M.gguf'
drive_folder = '/content/drive/My Drive/ModelFolder/'

shutil.move(model_path, drive_folder)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/ModelFolder/unsloth.Q4_K_M.gguf'

In [17]:
import json

# ✅ Convert the dataset to a list
eval_data_list = eval_dataset.to_list()

# ✅ Define the save path in Google Drive
eval_dataset_path = "/content/drive/MyDrive/eval_dataset.json"

# ✅ Save as JSON
with open(eval_dataset_path, "w", encoding="utf-8") as f:
    json.dump(eval_data_list, f, indent=4, ensure_ascii=False)

print(f"✅ Evaluation dataset saved at: {eval_dataset_path}")


✅ Evaluation dataset saved at: /content/drive/MyDrive/eval_dataset.json


# Obtaining Sample Responses from the Finetuned Model For the evaluation dataset Instructions

In [18]:
from google.colab import drive
import json

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Load Evaluation Dataset
eval_dataset_path = "/content/drive/MyDrive/eval_dataset.json"

with open(eval_dataset_path, "r", encoding="utf-8") as file:
    eval_data = json.load(file)

print(f"✅ Loaded {len(eval_data)} evaluation samples.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded 115 evaluation samples.


In [19]:
# ✅ Define the Alpaca Prompt Format
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token


In [20]:
# ✅ Generate responses for each evaluation sample
for item in eval_data:
    instruction = item["Instruction"]

    # ✅ Format the input prompt
    inputs = tokenizer(
        [alpaca_prompt.format(instruction, "", "")],  # Empty input/output
        return_tensors="pt"
    ).to("cuda")

    # ✅ Generate response
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # ✅ Clean up response
    response = response.replace(EOS_TOKEN, "").strip()

    # ✅ Store the generated response in the dataset
    item["Generated_Response"] = response

print("✅ Responses generated for all evaluation samples!")


✅ Responses generated for all evaluation samples!


In [21]:
# ✅ Define save path
eval_results_path = "/content/drive/MyDrive/eval_results.json"

# ✅ Save the updated dataset
with open(eval_results_path, "w", encoding="utf-8") as file:
    json.dump(eval_data, file, indent=4, ensure_ascii=False)

print(f"✅ Evaluation results saved at: {eval_results_path}")


✅ Evaluation results saved at: /content/drive/MyDrive/eval_results.json
